# 單元 07-2. 使用 CNN 建立 Variational Autoencoder

本單元的練習中，我們將帶領各位練習建立 CNN 版本的 VAE。

## 1. 初始準備

In [ ]:
%env KERAS_BACKEND=tensorflow

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Keras functions
from keras.models import Sequential
from keras.layers import Input, Activation, Lambda, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

from keras import backend as K

讀取 MNIST 手寫辨識資料

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## 2. 建立 VAE 的初始準備

In [ ]:
x = Input(shape=(28, 28, 1))

In [ ]:
enc_1 = Conv2D(2, 3, activation='relu')
enc_2 = Conv2D(2, 3, activation='sigmoid')

z_mean = GlobalAveragePooling2D()(enc_1(x))
z_log_var = GlobalAveragePooling2D()(enc_2(x))

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(2,), mean=0.,
                              stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [ ]:
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])

In [ ]:
dec_x = dec_2(z)
x_reconstructed = dec_1(dec_x)

In [ ]:
VAE = Model(x, x_reconstructed)
VAE.summary()

### 3.3 Loss 函數的建立
VAE 的 loss 函數，其由來牽扯一些訊息理論 (information theory) 的知識，因此，我們在此直接建立訓練 VAE 時的 loss 函數。

若對 VAE 的理論及模型基本設定有興趣的同學，可以參考下列兩篇論文：
* Auto-Encoding Variational Bayes: https://arxiv.org/pdf/1312.6114.pdf
* Tutorial on Variational Autoencoders: https://arxiv.org/pdf/1606.05908.pdf

In [ ]:
def vae_loss(x, x_recon):  
    
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

### 3.4 訓練 VAE

In [ ]:
VAE.compile(loss=vae_loss, optimizer='Nadam')

In [ ]:
VAE.fit(x_train, x_train, batch_size=32, epochs=100)

In [ ]:
# VAE.save_weights('VAE_handwriting_model_weights.h5')
VAE.load_weights('VAE_handwriting_model_weights.h5')

### 3.5 VAE 的視覺化呈現
與視覺化 Autoencoder 時的方式一樣，我們先分別定義出 Encoder 和 Decoder。

In [ ]:
VAE_Encoder = Model(x, z_mean)

VAE_Encoder.summary()

In [ ]:
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))

VAE_Decoder.summary()

首先，我們進行 Encoder 的視覺化呈現。

In [ ]:
idx = np.random.randint(x_train.shape[0])
print("第 %d 圖的 latent 表示為 %s" %(idx, VAE_Encoder.predict(x_train[idx: idx+1])))

In [ ]:
# idices = np.random.randint(x_test.shape[0], size=1000)
VAE_latents = VAE_Encoder.predict(x_test[idices])

In [ ]:
plt.scatter(VAE_latents[:, 0], VAE_latents[:, 1], c=y_test[idices])
plt.colorbar()
plt.show()

接著，我們進行 Decoder 的視覺化呈現。

In [ ]:
def normalize_to_unit(x):
    x -= x.min()
    x /= x.max()
    return x

In [ ]:
grid_x_vae = np.linspace(-4+0.05, 4-0.05, n)
grid_y_vae = np.linspace(-4+0.05, 4-0.05, n)
VAE_figure = np.zeros((digit_size * n, digit_size * n))
for i, yi in enumerate(grid_x_vae):
    for j, xi in enumerate(grid_y_vae):
        z_sample = np.array([[xi, yi]])
        x_decoded = VAE_Decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        VAE_figure[(n-i-1) * digit_size: (n - i) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = normalize_to_unit(digit)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(VAE_figure, cmap='Greys_r')
plt.axis('off')
plt.show()

### 3.6 與 Autoencoder 的 Decoder 視覺化進行比較

In [ ]:
plt.subplot(1, 2, 1)
plt.scatter(VAE_latents[:, 0], VAE_latents[:, 1], c=y_test[idices])
plt.colorbar()

plt.subplot(1, 2, 2)
plt.scatter(latents[:, 0], latents[:, 1], c=y_test[idices])
plt.colorbar()
plt.show()

## 4. 恭喜你，完成學習並建立 Variational Autoencoder 及神經網路的基本視覺化呈現。

Variational Autoencoder (VAE) 是一個重要的非監督式學習模型，具體應用的場合為特徵抽取/資料壓縮及還原，為影像處理中常見的模型之一。

恭喜各位已經完成了學習，在本單元的練習中，我們希望各位嘗試建立並訓練屬於自己的 VAE。